In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import pickle

In [2]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df=df.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [4]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [5]:
#one hot encode 'Geography' column
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [6]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
#combine one hot encoded columns with the original data 
df=pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [8]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [9]:
#split the data into features and target 
X=df.drop('EstimatedSalary', axis=1)
y=df['EstimatedSalary']

In [10]:
#split data into train and test sets 
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [11]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [13]:
#save the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender=pickle.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    label_encoder_geography=pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler=pickle.load(f)


ANN REGRESSION PROBLEM STATEMENT

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
#Build the model 
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
]) 

#compile the model 
model.compile(optimizer='adam', loss='mean_absolute_error', metrics='mae')

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
#set up early stopping 
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [21]:
history=model.fit(X_train_scaled, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 100381.9531 - mae: 100381.9531 - val_loss: 3009484.7500 - val_mae: 3009484.7500
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99701.1719 - mae: 99701.1719 - val_loss: 24015430.0000 - val_mae: 24015430.0000
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 97138.9453 - mae: 97138.9453 - val_loss: 84250432.0000 - val_mae: 84250432.0000
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 91942.5312 - mae: 91942.5312 - val_loss: 194981120.0000 - val_mae: 194981120.0000
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 84348.5391 - mae: 84348.5391 - val_loss: 359141664.0000 - val_mae: 359141664.0000
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 75307.7031 - mae: 75307.7031 - val_loss: 572427904.0000 - val_mae: 572427904.0000
Epoch 7/100
250/250 [==============================] - 2s 8ms/

In [23]:
#evaluate model on test data 
test_loss, test_mae=model.evaluate(X_test_scaled, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 1s 4ms/step - loss: 98537.6094 - mae: 98537.6094
Test MAE: 98537.609375


In [24]:
model.save('regression_model.h5')

c:\Users\nadee\Documents\ann\tfenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
